In [1]:
import os
import requests
import json
import csv
import re
import pandas as pd
from time import sleep
import datetime

In [21]:
print(os.path.abspath(os.getcwd()))
print(os.getcwd())
os.path.dirname(os.getcwd())
df = pd.read_csv(os.path.join(os.path.abspath(os.getcwd()),'stockName/zz500_stocks.csv'), encoding='gbk')
df.tail(1).updateDate.values[0]

d:\个人\Python学习\myspace\stockPick
d:\个人\Python学习\myspace\stockPick


'2022-01-17'

In [9]:
def get_file_info(self, file_dir):
    for root, dirs, files in os.walk(file_dir):
        # print(root)
        # print(dirs)
        # print(files)
        pass
    return root, dirs, files

In [4]:
'''datetime.datetime.now()-datetime.timedelta(days=1)
with open('t.csv', 'ab+') as w:
    w.write('\n'.encode('utf8'))
    w.write('??'.encode('utf8'))
w.close()'''

stockhisData_dir = r'C:\Users\chenc\OneDrive - whut.edu.cn\个人\AutoStock\dataDownload\sh600000.csv'
os.path.exists(stockhisData_dir)

True

In [ ]:
with open('b.csv', 'a+') as w:
    w.write('haha')
w.close()

with open('b.csv', 'a+') as w:
    w.write('haha')
w.close()
